In [1]:
#import necessary libraries
import os
from collections import Counter
import cv2
import numpy as np
import pandas as pd
import random
import math
from sklearn.preprocessing import FunctionTransformer
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
import albumentations as A
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import svm



## Explanation: Declare Variables, Create Augmented Folder, and Delete Existing Augmented Files

This code block performs the following steps:

1. **Declare Variables**:
    - `dataset_path`: The root directory where your dataset is stored.
    - `classes`: A list of class names (categories) in your dataset.
    - `img_size`: The target size (width, height) for images.
    - `target_size`: The desired number of images per class after augmentation.

2. **Create Augmented Folder for Each Class**:
    - For each class in `classes`, it constructs the path to an `augmented` subfolder inside the class directory.
    - If the `augmented` folder does not exist, it is created using `mkdir(parents=True, exist_ok=True)`.

3. **Delete All Existing Files in Augmented Folders**:
    - Iterates through all files in each `augmented` folder.
    - If a file is found, it is deleted using `unlink()`.
    - This ensures that the augmented folders are empty before new augmented images are generated.

In [2]:
#declare variables, Create augmented folder and delete Delete all existing augmented files inside augmented directories
dataset_path = "../dataset"
classes = ["glass", "paper", "cardboard", "plastic", "metal", "trash"]
img_size = (32, 32)
target_size=500
#delete existing augmented files
dataset_path = Path(dataset_path)  # make sure it's a Path object

for cls in classes:
    aug_dir = dataset_path  / cls/ "augmented"

    # Create folder if not exists
    aug_dir.mkdir(parents=True, exist_ok=True)

    # Delete all existing files inside
    for file in aug_dir.iterdir():
        if file.is_file():
            file.unlink()

## Explanation: split dataset into train and test sets


In [100]:
# split dataset into train and test sets
all_files = []
all_labels = []

for idx, cls in enumerate(classes):
    class_folder = os.path.join(dataset_path, cls)
    if not os.path.isdir(class_folder):
        print(f"Warning: Folder not found: {class_folder}")
        continue

    for f in os.listdir(class_folder):
        if not (f.endswith(".jpg") or f.endswith(".png")):
            continue
        file_path = os.path.join(class_folder, f)

        img = cv2.imread(file_path)
        if img is None:
            print(f"Warning: Cannot open image {file_path}, skipping.")
            continue

        all_files.append(file_path)
        all_labels.append(idx)

train_files, test_files, train_labels, test_labels = train_test_split(
    all_files, all_labels, test_size=0.35, random_state=42, stratify=all_labels
)

print(f"Train: {len(train_files)} images")
print(f"Test: {len(test_files)} images")


Train: 1212 images
Test: 653 images


# Image Augmentation Pipeline Explanation

This code defines an image augmentation pipeline using the **Albumentations** library.

---
## varible 
p is the probability to use this filtter and
n is the number of filtter that will choose randmly on each image


## Randomness for Different Results
To ensure each input image gets **different random augmentations every time**, we reset the random seeds:

```python
random.seed(None)
np.random.seed(None)



In [101]:
# Define augmentation pipeline
random.seed(None)
np.random.seed(None)

transform = A.Compose([
    A.SomeOf([
        A.HorizontalFlip(p=0.7),
        A.VerticalFlip(p=.7),
        A.Rotate(limit=50, p=.8),
        A.RandomScale(scale_limit=0.2, p=.5),
        A.RandomBrightnessContrast(p=0.5),
        A.GaussianBlur(p=0.3),
        A.GaussNoise(p=0.1),
        A.ColorJitter(p=1),
        A.CenterCrop(width=128, height=128, p=.1),
        A.Resize(width=random.randint(img_size[0], 256), height=random.randint(img_size[1], 256), p=.8)
    ], n=4)
])


## Function to perform data augmentation

In [ ]:
# Function to perform data augmentation
def data_autgmentation(img_path, output_folder, aug_times=5):
    img_name = os.path.basename(img_path)
    img = cv2.imread(img_path)
    if img is None:
        print(f"ERROR: Cannot load image {img_path}. Skipping.")
        return
    for i in range(aug_times):
        augmented = transform(image=img)
        aug_img = augmented["image"]
        aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i+1}.jpg"
        aug_img_path = os.path.join(output_folder, aug_img_name)
        cv2.imwrite(aug_img_path, aug_img)
    print(f"Augmented {aug_times} images for {img_name}")
    


In [ ]:
# Augment training images by 500 images per class
class_counts = Counter(train_labels)

for cls_idx, cls_name in enumerate(classes):
    
    cls_images = [img for i, img in enumerate(train_files) if train_labels[i] == cls_idx]
    current_count = class_counts[cls_idx]
    aug_needed = max(0, target_size - current_count)
    
    if aug_needed == 0:
        continue
    
    output_folder = os.path.join(dataset_path, cls_name, "augmented")
        
    times_per_image = math.ceil(aug_needed / len(cls_images))
    count=0
    for img_path in tqdm(cls_images, desc=f"Augmenting {cls_name}"):
        if count>= aug_needed:
            break
        data_autgmentation(img_path, output_folder, times_per_image)
        count=count+times_per_image



Augmenting glass:   0%|          | 0/250 [00:00<?, ?it/s]

✅ Augmented 1 images for 6763da26-9208-4935-b33b-7ad0ba56a406.jpg
✅ Augmented 1 images for 46e0469a-c98c-4e87-9a53-5fe50eca7611.jpg
✅ Augmented 1 images for cada275c-5e48-48a5-9eb0-1be6b68316ea.jpg
✅ Augmented 1 images for 0badb460-d33c-4a67-8fc7-3a62de33d30b.jpg
✅ Augmented 1 images for 6cf50200-2c63-471a-81b9-03778553ee60.jpg
✅ Augmented 1 images for a3810deb-d4e0-4546-8b4d-52e278ecaaf6.jpg
✅ Augmented 1 images for d51f4539-7e10-4c40-933b-35d9adaa5bd7.jpg
✅ Augmented 1 images for 50e5bbbf-b492-4250-a97c-4dac70b06d6a.jpg
✅ Augmented 1 images for 8e90dd34-e189-432d-8741-1dfd6d7858df.jpg
✅ Augmented 1 images for 17cd4044-9e8f-4082-861c-d8ca694f83af.jpg
✅ Augmented 1 images for c4990fc5-c8f5-4901-a655-6b372abffff2.jpg


Augmenting glass:   5%|▍         | 12/250 [00:00<00:02, 98.08it/s]

✅ Augmented 1 images for f23fb4d7-dd84-49d2-bf23-b51ce806c589.jpg
✅ Augmented 1 images for a999bc1a-838a-40bc-98a2-a0e90602791b.jpg
✅ Augmented 1 images for bee26d69-38fe-4f07-8ed0-9ac6530c2b2e.jpg
✅ Augmented 1 images for a0db09b3-0862-40d2-82b9-bde01ab2e750.jpg
✅ Augmented 1 images for 92036730-8118-420a-93d4-097c78d62082.jpg
✅ Augmented 1 images for a4e6c3d5-eb9d-4ba6-8537-bff9e2821d2b.jpg
✅ Augmented 1 images for 72d4ea84-5c06-40af-8efe-bfd10f088b53.jpg
✅ Augmented 1 images for 259cc3b2-e2e6-4a9b-bb27-9627fca2b124.jpg
✅ Augmented 1 images for 42e83751-4a5a-4da0-9038-3957b48816da.jpg
✅ Augmented 1 images for 3349a0d9-968b-4c20-a222-6464b8a063a5.jpg
✅ Augmented 1 images for 2ef87519-0a5b-4dc1-b054-2857c0426d79.jpg
✅ Augmented 1 images for f9e6e7cc-a81e-4259-a184-2b61622a5644.jpg
✅ Augmented 1 images for d25b7067-d140-47e5-9138-5be697b192b8.jpg
✅ Augmented 1 images for 94148dcc-3f7b-4575-99de-6be781105fbf.jpg
✅ Augmented 1 images for 71f55864-a701-4760-9d6e-28f205ddb095.jpg
✅ Augmente

Augmenting glass:  12%|█▏        | 29/250 [00:00<00:01, 134.97it/s]

...

✅ Augmented 7 images for f3b9fe67-29ee-42b8-bac4-f53ff209a4ce.jpg
✅ Augmented 7 images for b6c5cd66-33c2-4d24-943c-61cf0a6dbf0f.jpg
✅ Augmented 7 images for a7ab9752-6ea6-42ca-a46f-0bdd8745257d.jpg
✅ Augmented 7 images for 8ebbe8b0-4e0d-4093-a00c-40dc83aa6854.jpg
✅ Augmented 7 images for a8893fe7-02f1-48e3-90cc-bb24284e5d2e.jpg
✅ Augmented 7 images for f431c4ea-e2b1-41b3-a406-8f5d84cda488.jpg
✅ Augmented 7 images for 350fd2ac-0c27-4116-86ef-bba67af8302b.jpg


Augmenting trash:  84%|████████▍ | 58/69 [00:02<00:00, 26.89it/s]

✅ Augmented 7 images for d3fcfa27-7fd9-4e7a-b615-75fd0c6d3ec6.jpg
✅ Augmented 7 images for f0f82aac-35ba-46be-8aff-0acc56e55a30.jpg
✅ Augmented 7 images for 4642929b-ba87-4837-b2e8-a7313fbd4c9f.jpg
✅ Augmented 7 images for 13bf8cf0-7c3a-4dd3-814b-8224dbfb3e72.jpg
✅ Augmented 7 images for 4ecab47b-5ebd-4cf9-b27a-75e200a0cdcb.jpg
✅ Augmented 7 images for 41d8b40a-45cf-4eff-be62-bfa4d1fe37ed.jpg


Augmenting trash:  90%|████████▉ | 62/69 [00:02<00:00, 27.94it/s]

✅ Augmented 7 images for 86e46e96-db2c-4144-a36a-d09fcf5ac238.jpg
✅ Augmented 7 images for ba9fb748-e09e-4bcb-9aa0-a988dc679445.jpg


## for all augmented images, copy them to train_files and train_labels


In [104]:
# for all augmented images, copy them to train_files and train_labels
for cls in classes:
    aug_folder = os.path.join(dataset_path, cls, "augmented")
    if not os.path.isdir(aug_folder):
        continue
    for f in os.listdir(aug_folder):
        if not (f.endswith(".jpg") or f.endswith(".png")):
            continue
        file_path = os.path.join(aug_folder, f)
        train_files.append(file_path)
        train_labels.append(classes.index(cls))

# preprocessing

In [129]:
# Function to compute color histogram for an RGB image
def color_histogram(img, bins=(8, 8, 8)):
    """
    img: RGB image (H x W x 3)
    bins: number of bins for each channel (R, G, B)
    returns: flattened histogram (normalized)
    """
    hist = cv2.calcHist([img], [0, 1, 2], None, bins,
                        [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [319]:
from sklearn.decomposition import PCA

pca = PCA(n_components=140)   # choose number of components
lebel_encoder = LabelEncoder()
scaler = MinMaxScaler()


In [320]:
X_train=[]
for img_path in train_files:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    features = color_histogram(img)
    X_train.append(features)


X_train = np.array(X_train)
X_train=scaler.fit_transform(X=X_train)
X_train = pca.fit_transform(X_train)

y_train = np.array(train_labels)
y_train = lebel_encoder.fit_transform(y_train)


In [321]:
X_test=[]
for img_path in test_files:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    features = color_histogram(img)
    X_test.append(features)


X_test = np.array(X_test)
X_test=scaler.transform(X=X_test)
X_test=pca.transform(X_test)
y_test = np.array(test_labels)
y_test = lebel_encoder.transform(y_test)

## SVM

In [ ]:

svm_clf = svm.SVC(kernel='rbf', C=5, gamma='scale')

svm_clf.fit(X_train,y_train)

NameError: name 'prin' is not defined

In [428]:
y_svm_train_pred = svm_clf.predict(X_train)
accuracy = accuracy_score(y_train, y_svm_train_pred)
print("SVM train Accuracy:", accuracy)

SVM train Accuracy: 0.9025282767797738


In [429]:
y_svm_test_pred = svm_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_svm_test_pred)
print("SVM test Accuracy:", accuracy)

SVM test Accuracy: 0.6952526799387443


## K-NN

In [352]:
# Train KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [353]:
y_knn_train_pred = knn.predict(X_train)
accuracy = accuracy_score(y_train, y_knn_train_pred)
print("KNN Train Accuracy:", accuracy)

KNN Train Accuracy: 0.7538256819693946


In [354]:
y_knn_test_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_knn_test_pred)
print("KNN Train Accuracy:", accuracy)

KNN Train Accuracy: 0.667687595712098


# anther try

In [ ]:
pre = ImagePreprocessor(img_size=img_size, to_gray=False)
X_img = pre.transform(train_files)  # (N, H, W, C)

# 2. Flatten
X_flat = X_img.reshape(X_img.shape[0], -1)

# 3. Convert to DataFrame
pixel_cols = [f"pixel_{i}" for i in range(X_flat.shape[1])]

df = pd.DataFrame(X_flat, columns=pixel_cols)
df["label"] = train_labels


In [ ]:
lebel_encoder = LabelEncoder()
df['label'] = lebel_encoder.fit_transform(df['label'])

In [ ]:
df.head()

In [ ]:
df.drop_duplicates

In [130]:


class ImagePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, img_size=(256, 256), to_gray=False):
        self.img_size = img_size
        self.to_gray = to_gray

    def fit(self, X, y=None):
        return self  # No fitting needed

    def transform(self, X, y=None):
        processed = []
        for img_path in X:
            img = cv2.imread(img_path)
            if img is None:
                raise ValueError(f"Cannot read image: {img_path}")
            
            img = cv2.resize(img, self.img_size)
            
            if self.to_gray:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = np.expand_dims(img, axis=-1)  # keep channel dimension

            img = img.astype('float32') / 255.0  # Normalize to [0, 1]
            processed.append(img)

        return np.array(processed)


In [ ]:
pre = ImagePreprocessor(img_size=img_size, to_gray=False)
X_img = pre.transform(test_files)  # (N, H, W, C)

# 2. Flatten
X_flat = X_img.reshape(X_img.shape[0], -1)

# 3. Convert to DataFrame
pixel_cols = [f"pixel_{i}" for i in range(X_flat.shape[1])]

df_test = pd.DataFrame(X_flat, columns=pixel_cols)
df_test["label"] = test_labels


In [ ]:
X_test = df_test.drop("label", axis=1)
y_test = df_test["label"]


In [ ]:
X = df.drop("label", axis=1)
y = df["label"]


In [ ]:
#train svm classifier
from sklearn import svm
svm_clf = svm.SVC(kernel='rbf', C=5, gamma='scale')
svm_clf.fit(X, y)
y_svm_pred = svm_clf.predict(X)

In [ ]:
accuracy = accuracy_score(df['label'], y_svm_pred)
print("Accuracy:", accuracy)

In [ ]:
y_test_svm_pred = svm_clf.predict(X_test)
accuracy = accuracy_score(df_test['label'], y_test_svm_pred)
print("Test Accuracy:", accuracy)

In [ ]:
# Train KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, y)
y_pred = knn.predict(X)

In [ ]:

accuracy = accuracy_score(df['label'], y_pred)
print("Accuracy:", accuracy)

In [ ]:


pipeline = Pipeline([
    ('preprocess', ImagePreprocessor(img_size=img_size, to_gray=False)),
    ('flatten', FunctionTransformer(flatten)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])


In [ ]:
y_pred = pipeline.predict(df['image_path'])

In [ ]:

accuracy = accuracy_score(df['image_label'], y_pred)
print("Accuracy:", accuracy)


In [ ]:
df_test = pd.DataFrame({
    'image_path': test_files,
    'image_label': test_labels
})
df_test['image_label'] = lebel_encoder.transform(df_test['image_label'])
y_pred = pipeline.predict(df_test['image_path'])

In [ ]:


cm = confusion_matrix(df_test['image_label'], y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
